<a href="https://colab.research.google.com/github/IlTACK-OH/pandas_practice/blob/main/day5/11_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 집계하기 - groupby 메서드
`groupby`메서드로 평균값을 구하는 과정을 통해 데이터 집계가 무엇인지 알아보겠다.

# groupby 메서드로 평균값 구하기

In [1]:
# 갭마인더 집합 불러오기
import pandas as pd

In [2]:
from google.colab import files
files.upload();

Saving gapminder.tsv to gapminder.tsv


In [3]:
df = pd.read_csv('gapminder.tsv', sep = '\t')

`year`열을 기준으로 데이터를 그룹화한 다음 `lifeExp`열의 평균을 구하여 보겠다.

In [4]:
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
avg_life_exp_by_year

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

# 분할-반영-결합 과정 살펴보기

앞에서 `groupby`메서드를 사용해 `lifeExp`열의 평균값을 구했다.<br>`groupby`메서드 자체를 분해하여 살펴보는 것은 불가능하기 때문에<br> 비슷한 연산을 수행하는 메서드를 순서대로 실행하며 알아보겠다.

---
`groupby`메서드에 열 이름을 전달하면 `분할`작업이 먼저 일어난다.

In [5]:
years = df.year.unique() # year 열의 데이터를 중복 없이 추출한 것이다.
years

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007])

다음으로 일단 각 연도별로 데이터를 추출하여 연도별로 평균값을 구한다.<br>이 과정을 `반영`작업의 한 부분이라고 생각하면 된다.

In [6]:
y1952 = df.loc[df.year == 1952,:]
y1952.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
12,Albania,Europe,1952,55.230,1282697,1601.056136
24,Algeria,Africa,1952,43.077,9279525,2449.008185
36,Angola,Africa,1952,30.015,4232095,3520.610273
48,Argentina,Americas,1952,62.485,17876956,5911.315053


위 데이터에서 `lifeExp` 열의 평균값을 구하여보겠다.<br>
이 과정도 `반영` 작업의 한 부분이다.

In [7]:
y1952_mean = y1952.lifeExp.mean()
y1952_mean

49.057619718309866

위 작업을 반복하여 남은 연도의 평균값을 구하면 비로소 `반영`작업이 끝난다.

In [8]:
y1957 = df.loc[df.year==1957,:]
y1957_mean = y1957.lifeExp.mean()
y1957_mean

51.50740112676056

In [9]:
y1962 = df.loc[df.year==1962,:]
y1962_mean = y1962.lifeExp.mean()
y1962_mean

53.609249014084504

In [10]:
y2007 = df.loc[df.year == 2007,:]
y2007_mean = y2007.lifeExp.mean()
y2007_mean

67.00742253521126

마지막으로 연도별로 계산한 lifeExp의 평균값을 합친다.<br>이 과정이 `결합`작업이다.

In [11]:
df2 = pd.DataFrame({'year':[1952,1957,1962,2007],"":[y1952_mean,y1957_mean,y1962_mean,y2007_mean]})
df2

,year,
0,1952,49.057620
1,1957,51.507401
2,1962,53.609249
3,2007,67.007423


# groupby 메서드와 함께 사용하는 집계 메서드
>**`count`**: 누락값을 제외한 데이터 수를 반환<br>
**`size`**: 누락값을 포함한 데이터 수를 반환<br>
**`mean`**: 평균값 반환<br>
**`std`**: 표준편차 반환<br>
**`min`**: 최솟값 반환<br>
**`quantile(q=0.25)`**: 백분위수 25%<br>
**`quantile(q=0.50)`**: 백분위수 50%<br>
**`quantile(q=0.75)`**: 백분위수 75%<br>
**`max`**: 최댓값 반환<br>
**`sum`**: 전체 합 반환<br>
**`var`**: 분산 반환<br>
**`sem`**: 평균의 표준편차 반환<br>
**`describe`**: 데이터 수, 평균, 표준편차, 최소값, 백분위수(25,50,75%), 최댓값을 모두 반환<br>
**`first`**: 첫 번째 행 반환<br>
**`last`**: 마지막 행 반환<br>
**`nth`**: n번째 행 반환<br>

# agg 메서드로 사용자 함수와 groupby메서드 조합하기
라이브러리에서 제공하는 집계 메서드로 원하는 값을 계산할 수 없는 경우에는 직접 함수를 만들어 사용해여 한다.<br>이번에는 사용자 함수와 `groupby`메서드를 조합해서 사용해보겠다.<br>사용자 함수와 `groupby`메서드를 조합하려면 `agg`메서드를 이용해야 한다.

# 평균값을 구하는 사용자 함수와 groupby 메서드

In [12]:
# 평균값 구하는 함수
def my_mean(values):
  n = len(values)
  sum =0
  for value in values:
    sum+=value
  return sum/n

In [13]:
# agg메서드를 사용
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
agg_my_mean

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

# 두 개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드

In [14]:
# 첫 번째 인자로 받은 열의 평균값을 구하여 두 번째 인자로 받은 값과의 차이를 계산하는 함수
def my_mean_diff(values,diff_value):
  n = len(values)
  sum =0
  for value in values:
    sum+=value
  mean = sum/n
  return mean-diff_value

In [15]:
# 연도별 평균 수명에서 전체 평균 수명을 뺀 값.
# agg메서드의 첫 번재 인자에 my_mean_diff함수를 전달하고
# 두 번째 인자에 전체 평균 수명값을 전달한다.
global_mean = df.lifeExp.mean()
global_mean

59.474439366197174

In [16]:
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff,diff_value=global_mean)
agg_mean_diff

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64

# 집계 메서드를 리스트, 딕셔너리에 담아 전달하기
여러 개의 집계 메서드를 한 번에 사용하고 싶을 경우, 집계 메서드를 리스트나 딕셔너리에 담아 `agg`메서드에 전달하면 된다.<br>

---
연도별로 그룹화한 lifeExp열의 0이 아닌 값의 개수, 평균, 표준편차를 한 번에 계산하여 출력해보겠다.<br> 넘파이 메서드인 `count_nonzero`,`mean`,`std`를 리스트에 담아 `agg`메서드에 전달하면 된다.

In [17]:
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero,np.mean,np.std])
gdf

,count_nonzero,mean,std
year,,,
1952,142,49.057620,12.225956
1957,142,51.507401,12.231286
1962,142,53.609249,12.097245
1967,142,55.678290,11.718858
1972,142,57.647386,11.381953
1977,142,59.570157,11.227229
1982,142,61.533197,10.770618
1987,142,63.212613,10.556285
1992,142,64.160338,11.227380


이번에는 집계 메서드를 딕녀서리에 담아 `agg`메서드에 전달해 보겠다.<br>딕셔너리의 키로 집계 메서드를 적용할 열 이름을 전달하고 딕셔너리의 값으로 집계 메서드를 전달하면 된다.

In [18]:
gdf_dict = df.groupby('year').agg({'lifeExp':'mean','pop':'median','gdpPercap':'median'})
gdf_dict

,lifeExp,pop,gdpPercap
year,,,
1952,49.057620,3943953.0,1968.528344
1957,51.507401,4282942.0,2173.220291
1962,53.609249,4686039.5,2335.439533
1967,55.678290,5170175.5,2678.334740
1972,57.647386,5877996.5,3339.129407
1977,59.570157,6404036.5,3798.609244
1982,61.533197,7007320.0,4216.228428
1987,63.212613,7774861.5,4280.300366
1992,64.160338,8688686.5,4386.085502


# 표준 점수 계산하기

In [19]:
# 표준점수를 계산하는 함수.
def my_zscore(x):
  return (x -x.mean())/x.std()

`my_zscore`함수를 적용하여 `lifeExp`열의 표준점수를 계산하기 위해 `transform`메서드를 사용하였다.

In [20]:
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
transform_z.head()

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64

`my_zscore`함수는 데이터를 표준화할 뿐 집계는 하지 않는다.<br>즉, 데이터의 양이 줄어들지 않는다.

In [21]:
df.shape

(1704, 6)

In [22]:
transform_z.shape

(1704,)

# 누락값을 평균값으로 처리하기

In [24]:
import seaborn as sns
import numpy as np
# seaborn 라이브러리의 tips데이터 집합에서 10개의 행 데이터만 가져온 다음
# totall_bill열의 값 4개를 임의로 선택하여 누락값으로 바꿔보겠다.
np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4],'total_bill'] = np.NaN
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,NaN,2.00,Male,No,Sun,Dinner,4
211,NaN,5.16,Male,Yes,Sat,Dinner,4
198,NaN,2.00,Female,Yes,Thur,Lunch,2
176,NaN,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


성별로 그룹화한 다음 각 열의 데이터 수를 구하여 보겠다.

In [25]:
count_sex = tips_10.groupby('sex').count()
count_sex

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,4,7,7,7,7,7
Female,2,3,3,3,3,3


성별을 구분하여 total_bill열의 데이터를 받아 평균값을 구하는 함수를 만들어보겠다.

In [26]:
def fill_na_mean(x):
  avg = x.mean()
  return x.fillna(avg)

성별을 구분한 `total_bill`열의 데이터를 `fill_na_mean`함수에 전달하여 평균값을 구한 다음 tips_10에 새로운 열로 추가하여보겠다.

In [27]:
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
tips_10

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
24,19.82,3.18,Male,No,Sat,Dinner,2,19.8200
6,8.77,2.00,Male,No,Sun,Dinner,2,8.7700
153,NaN,2.00,Male,No,Sun,Dinner,4,17.9525
211,NaN,5.16,Male,Yes,Sat,Dinner,4,17.9525
198,NaN,2.00,Female,Yes,Thur,Lunch,2,13.9300
176,NaN,2.00,Male,Yes,Sun,Dinner,2,17.9525
192,28.44,2.56,Male,Yes,Thur,Lunch,2,28.4400
124,12.48,2.52,Female,No,Thur,Lunch,2,12.4800
9,14.78,3.23,Male,No,Sun,Dinner,2,14.7800
101,15.38,3.00,Female,Yes,Fri,Dinner,2,15.3800


# 데이터 필터링 사용하기 ─ filter 메서드

In [28]:
tips = sns.load_dataset('tips')
tips.shape

(244, 7)

In [29]:
tips['size'].value_counts()

2    156
3     38
4     37
5      5
1      4
6      4
Name: size, dtype: int64

30번 이상의 주문이 있는 테이블만 추려 데이터 분석을 하기 위해서 다음과 같이 그룹화하여 새로운 변수에 저장하여보겠다.

In [32]:
tips_filtered = tips.groupby('size').filter(lambda x: x['size'].count() >= 30)

In [33]:
tips_filtered.shape

(231, 7)

1,5,6테이블의 데이터가 제외되어 5+4+4=13만큼 전체 행이 줄어든 것을 확인할 수 있다.

In [34]:
tips_filtered['size'].value_counts()

2    156
3     38
4     37
Name: size, dtype: int64

# 그룹 오브젝트 저장하여 살펴보기

In [35]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,24.55,2.00,Male,No,Sun,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
198,13.00,2.00,Female,Yes,Thur,Lunch,2
176,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [36]:
grouped = tips_10.groupby('sex')
grouped

groupby메서드의 결괏값을 출력하면 자료형이 그룹 오브젝트라는 것을 확인할 수 있다.<br><br>
그룹 오브젝트에 포함된 그룹을 보려면 `groups`속성을 출력하면 된다.<br>그러면 `sex`열로 그룹화한 데이터프레임의 인덱스를 확인할 수 있다.<br>
이 그룹 오브젝트로 집계, 변환, 필터 작업을 수행하면 된다.

In [37]:
grouped.groups

{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}

# 그룹 오브젝트의 평균 구하기

평균값을 구할 수 없는 열에 집계 메서드를 사용하여도 오류가 발생하지 않는다.<br>파이썬은 자동으로 계산할 수 있는 열을 골라주는 기능을 제공하기 때문에<br>그룹 오브젝트에 `mean`메서드를 사용해도 바로 평균값을 구할 수 있다.

In [38]:
avgs = grouped.mean()
avgs

,total_bill,tip,size
sex,,,
Male,20.02,2.875714,2.571429
Female,13.62,2.506667,2.000000


In [39]:
tips_10.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

# 그룹 오브젝트에서 데이터 추출하고 반복하기

만약 그룹 오브젝트에서 특정 데이터만 추출하려면 `get_group`메서드를 사용하면 된다.

In [40]:
female = grouped.get_group('Female')
female

,total_bill,tip,sex,smoker,day,time,size
198,13.00,2.00,Female,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [41]:
for sex_group in grouped:
  print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


sex_group의 자세한 정보를 출력해보겠다.

In [45]:
for sex_group in grouped:
  print('the type is: {}\n'.format(type(sex_group)))
  print('the length is: {}\n'.format(len(sex_group)))

  first_element = sex_group[0]
  print('the first element is: {}\n'.format(first_element))
  print('it has a type of: {}\n'.format(type(sex_group[0])))

  second_element = sex_group[1]
  print('the second element is:\n{}\n'.format(second_element))
  print('it has a type of: {}\n'.format(type(second_element)))

  print('what we have:')
  print(sex_group)

  break

the type is: <class 'tuple'>

the length is: 2

the first element is: Male

it has a type of: <class 'str'>

the second element is:
     total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2

it has a type of: <class 'pandas.core.frame.DataFrame'>

what we have:
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  

# 그룹 오브젝트 계산하고 살펴보기

In [46]:
bill_sex_time = tips_10.groupby(['sex','time'])
group_avg = bill_sex_time.mean()

group_avg

total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000

In [47]:
print(type(group_avg))

<class 'pandas.core.frame.DataFrame'>


In [48]:
print(group_avg.columns)

Index(['total_bill', 'tip', 'size'], dtype='object')


In [49]:
print(group_avg.index)

MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


위와 같이 데이터프레임의 인덱스가 `MultiIndex`인 경우에는 `reset_index`메서드를 사용하여<br> 데이터프레임의 인덱스를 새로 부여할 수 있다.

In [50]:
group_method = tips_10.groupby(['sex','time']).mean().reset_index()
group_method

,sex,time,total_bill,tip,size
0,Male,Lunch,28.440000,2.560000,2.000000
1,Male,Dinner,18.616667,2.928333,2.666667
2,Female,Lunch,12.740000,2.260000,2.000000
3,Female,Dinner,15.380000,3.000000,2.000000


`as_index`인자를 False로 설정해도 같은 결과를 얻을 수 있다.

In [51]:
group_param = tips_10.groupby(['sex','time'],as_index = False).mean()
group_param

,sex,time,total_bill,tip,size
0,Male,Lunch,28.440000,2.560000,2.000000
1,Male,Dinner,18.616667,2.928333,2.666667
2,Female,Lunch,12.740000,2.260000,2.000000
3,Female,Dinner,15.380000,3.000000,2.000000
